<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Bus_Route_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assign Bus Companies to School Routes

## Objective and Prerequisites

This school bus route assignments problem shows you how to determine the optimal assignment of companies to bus routes, where each company may not bid on all the routes. The objectives of the assignment problem are:

* Make sure each route is served by a bus company,
* Minimize the overall operating cost of covering all the routes, and
* Ensure that the assignments are valid, i.e., cannot assign a company to a route if the company does not bid on the route.

This modeling example is at the introductory level, where we assume that you know Python and that you have some knowledge of how to build mathematical optimization models.

---
## Problem Description

![picture](https://drive.google.com/uc?id=1qQFIoW3dQCbS2lwSYpix3m738nhACzMX)

The city of Spring View is taking bids from six bus companies on the eight routes that must be driven in the surrounding school district. Each company enters a bid of how much it will charge to drive selected routes, although not all companies bid on all routes.  

The following table lists the bids by each company on different routes. Note that not all companies bid on all routes - this is a critical new elements we are going to model.

| Company |	1	    | 2	    | 3 	  | 4	   | 5	  | 6	   | 7	  | 8    |
| ---     | ---   | ---   | ---   | ---  | ---  | ---  | ---  | ---  |
|1	      | -	    | 8200	| 7800	| 5400 | -	  | 3900 | -	  | -    |
|2	      | 7800	| 8200	|-	    | 6300 |	-	  | 3300 | 4900 |	-    |
|3	      | -     |	4800	|-	    |-	   |-	    | 4400 | 5600 |	3600 |
|4	      | -	    | -	    |8000   |	5000 | 6800 |	-	   | 6700 |	4200 |
|5 	      | 7200	| 6400	|-	    | 3900 | 6400	| 2800 | -	  | 3000 |
|6	      | 7000	| 5800	|7500	  | 4500 | 5600	|-	   | 6000	| 4200 |

If a company does not bid on a route, the corresponding entry is blank. The city must decide which companies to assign to which routes with the specifications that if a company does not bid on a route, it cannot be assigned to that route; exactly one company must be assigned to each route; and
a company can be assigned to at most two routes. The objective is to minimize the total cost of covering all routes.

We would like to find a most efficient assignment plan to minimize the cost of covering all routes. This example shows how to use sparse representation to model the fact that not all companies bid on all routes.

## Model Formulation

### Indices

$i \in \{1..6\}$: Index of six different bus companies.

$j \in \{1..8\}$: Index of eight different routes.

### Parameters

$A$: Set of tuples ($i,j$) where company $i$ has a bid on route $j$.

$c_{ij}$: Bid by company $i$ on route $j$.


### Decision Variables

$x_{ij}$: Whether to assign company $i$ to route $j$.


### Objective Function

- **Cost**. We want to minimize the total operating cost.


\begin{equation}
\text{Min}_{x_{ij}} \quad \sum_{(i,j) \in A} c_{ij}*x_{ij}
\tag{0}
\end{equation}

### Constraints

- **Staffing Requirement**. Satisfy staffing requirement for each day.

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} \geq 1 \quad \forall j \in \{1...8\} \quad (\text{each route must be covered})
\tag{1}
\end{equation}

\begin{equation}
\sum_{j|(i,j) \in A} x_{ij} \leq 2 \quad \forall i \in \{1...6\} \quad (\text{each company cannot be assigned more than two routes})
\tag{2}
\end{equation}

\begin{equation}
x_{ij} \in \{0,1\} \quad \forall (i,j) \in A \quad (\text{binary assignments})
\tag{3}
\end{equation}

---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 8.7 MB/s 


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

Set up the model and solve

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('bus routes')

# indices for companies and routes
company = [*range(0,6)]
route = [*range(0,8)]

# bids 
c = [[0,	8200,	7800,	5400,	0,	3900,	0,	0],
    [7800,	8200,	0,	6300,	0,	3300,	4900,	0],
    [0,	4800,	0,	0,	0,	4400,	5600,	3600],
    [0,	0,	8000,	5000,	6800,	0,	6700,	4200],
    [7200,	6400,	0,	3900,	6400,	2800,	0,	3000],
    [7000,	5800,	7500,	4500,	5600,	0,	6000,	4200]]

# Valid set of tuples
A = []
for i in company:
    for j in route:
        if c[i][j] > 0:
            tp = i,j
            A.append(tp)

# take a look at the set
# print(np.matrix(A))

# valid set of bids for route j
AJ = [] 
k = 0
for l in route:
    A_temp = []
    for i in company:
        for j in route:
            if c[i][j] > 0:
                if j==k:
                    tp = i,j
                    A_temp.append(tp)
    AJ.append(A_temp) 
    k+=1               

# take a look at a sample
# print(np.matrix(AJ[0]))

# valid set of routes that can be covered by company i
AI = [] 
k = 0
for l in route:
    A_temp = []
    for i in company:
        for j in route:
            if c[i][j] > 0:
                if i==k:
                    tp = i,j
                    A_temp.append(tp)
    AI.append(A_temp) 
    k+=1               

# take a look at a sample
# print(np.matrix(AI[0]))

# Build decision variables: where to assign company i to route j
x = m.addVars(A, vtype=GRB.BINARY, name='Assign')
    
# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(c[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)
    
# Cover each route
routeConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AJ[j]) >= 1 for j in route), 
                                      name='routeConstrs')

# No company can be assigned more than two routes
companyConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[i]) <= 2 for i in company), 
                                      name='companyConstrs')

# Run optimization engine
m.optimize()

Take a look at the results of assignments

In [ ]:
#####################################################
#         Assignment results
#####################################################
    
print(f"\n\n___Optimal assignment on each route________")
t_cost = 0
for i,j in A:
    if x[(i,j)].x > 0:
        print("Company %2d is assigned to route %2d: $%4d" % (i+1, j+1, c[i][j]))
        t_cost += c[i][j]

print("The total cost of covering all routes is $%5d" % (t_cost))
            

As we can see, all routes are covered by exactly one company, and no companies are assigned more than two routes. We observe that companies 2, 5, and 6 are assigned two routes while companies 1 and 3 are assigned one route. Further, companies are only assigned to routes where they have a valid bid. 

---
##  Conclusion

In this example, we addressed the but route assignment problem. We determined the optimal assignment of companies to routes: 
* Satisfy coverage for each route, 
* Minimize the total operating cost, and 
* Ensure no companies are assigned more than two routes, and no companies are assigned to a route where the is no bid.

A special technique in the model formulation is sparse reprentation, where we significantly reduce the number of decision variables by restricting the set of decisions to be on the valid bids only. This benefit becomes more significant as problem size grows.

This bus assignment model can be used by many organizations to help make informed decisions about covering certain demands from a diverse set of resources where not all resources are suitable for all demands.


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.

[2] Gurobi python reference. https://www.gurobi.com/documentation/

[3] This notebook is developed by Yimin Wang. If you have any comments or suggestions, please contact yimin_wang@asu.edu